In [0]:
%run /Workspace/Users/valterlafuentejunior@gmail.com/ProjetoMedalhao/00.config/config

In [0]:
%run /Workspace/Users/valterlafuentejunior@gmail.com/ProjetoMedalhao/Utils/utils


In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window


In [0]:
tabela_destino = "workspace.gold_etl.varicacao_ipca_boiGordo"


In [0]:
insights = spark.table("silver_etl.economia")

In [0]:
insights = insights.dropDuplicates(["data"])


In [0]:
# Criando Janela window ordenada pelo campo data_coleta
w = Window.orderBy("data")

In [0]:
insights = (
    insights
    .withColumn("ipca_anterior", F.coalesce(F.lag("ipca").over(w), F.lit(0)))
    .withColumn("boi_anterior", F.coalesce(F.lag("boi_gordo").over(w), F.lit(0)))
    .withColumn(
        "var_ipca",
        F.when(F.col("ipca_anterior") != 0,
               (F.col("ipca") - F.col("ipca_anterior")) / F.col("ipca_anterior") * 100
        ).otherwise(F.lit(0))
    )
    .withColumn(
        "var_boi",
        F.when(F.col("boi_anterior") != 0,
               (F.col("boi_gordo") - F.col("boi_anterior")) / F.col("boi_anterior") * 100
        ).otherwise(F.lit(0))
    )
    .drop("ipca_anterior", "boi_anterior")
)


In [0]:
merge_delta_table(insights, tabela_destino)